In [3]:
#!/usr/bin/env python3

from cimaq_utils import *
from cimaq_utils import loadimages
from blind_rename import *
from inspect_misc_text import *
from inspect_misc_text import no_ascii
from zipctl import uzipfiles
from get_infos import get_infos
from fix_cimaq import get_cimaq_dir_paths
# from get_zipinfos import get_zipinfos

from os.path import expanduser as xpu
from os.path import join as pjoin
from collections import OrderedDict
from tkinter import filedialog 
from zipfile import ZipFile
import codecs

from cimaq_filter import cimaq_filter
from fix_cimaq import get_cimaq_dir_paths
from zipctl import getnamelist
from zipctl import getinfolist
from chardet import UniversalDetector as Udet
# 'cimaq_derivatives/task_files/uzeprimes'

cimaq_dir="~/../../media/francois/seagate_1tb/cimaq_03-19/cimaq_derivatives"

In [4]:
print(79*'#')

###############################################################################


In [5]:
def get_cimaq_dir_paths(cimaq_dir="~/../../media/francois/seagate_1tb/cimaq_03-19/cimaq_derivatives"):
    json_params = json_read(xpu('~/cimaq_memory/cimaq_dir_list.json'), 'r')
    dlst = df.from_dict(json_params['dir_list'], orient='index', columns=['suffixes'])
    patterns =  df.from_dict(json_params['patterns'], orient='index', columns=['patterns'])
    dlst['fpaths'] = [join(xpu(cimaq_dir), sfx) for sfx in dlst.suffixes]
    return dlst.T, patterns


In [6]:
dlst, patterns = get_cimaq_dir_paths()
dlst

,confdir,mean_motion,mean_file,mean_npsych,mean_bids,mean_qc,mean_behav,new_mean_dir,uzeprimes,zeprimes,pdf_files,edat2_files
suffixes,confounds,participants/MotionResults/fMRI_meanMotion.tsv,participants/MemoTaskParticipantFile.tsv,participants/Neuropsych/ALL_Neuropsych_scores.tsv,participants/Participants_bids.tsv,participants/sub_list_TaskQC.tsv,participants/TaskResults/fMRI_behavMemoScores.tsv,mean_sheets,participants/task_files/uzeprimes,task_files/zipped_eprime,task_files/pdf_files,task_files/edat2_files
fpaths,/home/francois/../../media/francois/seagate_1t...,/home/francois/../../media/francois/seagate_1t...,/home/francois/../../media/francois/seagate_1t...,/home/francois/../../media/francois/seagate_1t...,/home/francois/../../media/francois/seagate_1t...,/home/francois/../../media/francois/seagate_1t...,/home/francois/../../media/francois/seagate_1t...,/home/francois/../../media/francois/seagate_1t...,/home/francois/../../media/francois/seagate_1t...,/home/francois/../../media/francois/seagate_1t...,/home/francois/../../media/francois/seagate_1t...,/home/francois/../../media/francois/seagate_1t...


In [7]:
def get_zipcontents(fpath):
    myzip = ZipFile(fpath, 'r')
    nmlst = tuple(getnamelist(fpath))
    contents = df(tuple(dict(zip(evenodd(itm)[0], evenodd(itm)[1])) for itm
                        in tuple(tuple(no_ascii(repr(itm)).strip().replace(
                   "'", "").replace("'",'').replace(
                       '=',' ')[:-2].split())[1:]
                   for itm in set(repr(myzip.getinfo(itm)).strip(
                       ' ').replace(itm, itm.replace(' ', '_')) if ' ' in
                                  itm else repr(myzip.getinfo(itm)).strip(
                                      ' ') for itm in nmlst))))
    contents[['dst_paths', 'exts']] = \
        [(join(os.path.splitext(fpath)[0], fnm), os.path.splitext(fnm)[1])
         for fnm in contents.filename]
    contents[['src_paths', 'src_names']] = \
        [(join(os.path.splitext(fpath)[0], fnm), fnm) for fnm in nmlst]
    contents = contents.sort_values('filename').reset_index(drop=True)    
    contents['bsheets'] = [myzip.open(row[1].src_names, 'r').read()
                           for row in tqdm(contents.iterrows())]
    myzip.close()
#     contents['encoding'] = [[chardet.detect(bsheet)['encoding'] if not None
#                              else 'cp437'] for bsheet in contents.bsheets]    
    return contents


In [140]:

def get_enc_zip(fpath):
    contents = get_zipcontents(fpath)
    for row in contents.iterrows():
        if row[1]['exts'] == '.pdf':
            enc = 'utf8'
        else:
            enc = ('cp437' if chardet.detect(row[1].bsheets)['encoding'] == None
                   else chardet.detect(row[1].bsheets)['encoding'])

#         enc = chardet.detect(row[1].bsheets)['encoding']
#     contents['encoding'] = ['utf8' if row[1].exts == '.pdf' else
#                             chardet.detect(row[1].bsheets)['encoding'] if 
#                             chardet.detect(row[1].bsheets)['encoding'] != None
#                             else 'cp437' for row in contents.iterrows()]
    return contents


In [606]:
# def strip_non_ascii(string):
#     ''' Returns the string without non ASCII characters'''
#     stripped = (c for c in string if 0 < ord(c) < 127)
#     return ''.join(stripped)

def get_zipcontents(fpath):
    myzip = ZipFile(fpath, 'r')
    nmlst = tuple(getnamelist(fpath))
    contents = df(tuple(dict(zip(evenodd(itm)[0], evenodd(itm)[1])) for itm
                        in tuple(tuple(no_ascii(repr(itm)).strip().replace(
                   "'", "").replace("'",'').replace(
                       '=',' ')[:-2].split())[1:]
                   for itm in set(repr(myzip.getinfo(itm)).strip(
                       ' ').replace(itm, itm.replace(' ', '_')) if ' ' in
                                  itm else repr(myzip.getinfo(itm)).strip(
                                      ' ') for itm in nmlst))))
    contents[['dst_paths', 'exts']] = \
        [(join(os.path.splitext(fpath)[0], fnm), os.path.splitext(fnm)[1])
         for fnm in contents.filename]
    contents[['src_paths', 'src_names']] = \
        [(join(os.path.splitext(fpath)[0], fnm), fnm) for fnm in nmlst]
    contents = contents.sort_values('filename').reset_index(drop = True)    
    contents['bsheets'] = [myzip.open(row[1].src_names, 'r').read()
                           for row in contents.iterrows()]
    myzip.close()
    return contents

def get_encoding_zip_singlefunc(fpath):
    encodings, lterms = [], []
    contents = get_zipcontents(fpath)
    for row in contents.iterrows():
        detector = udet()
        detector.reset()
        while True:
            next((detector.feed(line) for line in
                           row[1]['bsheets'].splitlines()))
            if not detector.done and not detector.result:
                continue
            else:
                break
            break
        detector.close()
        encodings.append((row[1]['filename'],
                          tuple(detector.result.values())[0]))
    contents['encoding'] = [itm[1] for itm in encodings]
    contents['lineterminator'] = [pd.Series([itm[0].strip(itm[1]) for itm in
                                  tuple(zip(row[1]['bsheets'].splitlines(keepends=True),
                                            row[1]['bsheets'].splitlines()))]).unique()
                                  for row in contents.iterrows()]
    contents['decoded'] = [row[1]['bsheets'].lower().decode(row[1]['encoding'],
                                                            'ignore').encode('utf8',
                                                                             'ignore').decode('utf8', 'relpace').strip('?')
#         row[1]['encoding'], 'ignore'),
#                                 'utf8', 'replace').decode('utf8').strip()
#         row[1]['encoding'],
#                                                     'ignore'), 'utf8').decode(
#                               'utf8', 'ignore')
                           for row in contents.iterrows()]
#     encs = (itm for itm in contents['encoding'].unique())
#     for row in contents.iterrows():
#         while True:


#             try:
#                 contents['decoded'] = row[1]['bsheets'].decode(next(encs))
#                                                 #.encode('utf8').decode('utf8'))
#                 contents['encod_OK'] = next(encs for row in contents.iterrows())

#             except UnicodeDecodeError:
#                 continue
#             else:
#                 break
#             break
    return contents



In [607]:
with_encs = [get_encoding_zip_singlefunc(fpath) for fpath in
             tqdm(loadimages(get_cimaq_dir_paths()[0].zeprimes.fpaths))]

100%|██████████| 103/103 [00:03<00:00, 26.90it/s]


In [605]:
with_encs[3].iloc[5].decoded

'category\tstim\toldnumber\trecognition_acc\trecognition_resp\trecognition_rt\tspatial_resp\tspatial_rt\tspatial_acc(à corriger voir output-encodage)\r\nold\tanimal_zebra_old.bmp\told13\t0\t2\t7197\t\t0\r\nold\tkitchen_big cooking pot.bmp\told28\t1\t1\t6459\t6\t1575\r\nold\tmusical_piano_old.bmp\told48\t1\t1\t2526\t9\t384\r\nnew\tmusical_zamponia windflute.bmp\tnew29\t1\t2\t1711\t9\t384\r\nnew\tanimal_ostrich.bmp\tnew26\t1\t2\t1944\t9\t384\r\nold\tvegie_cauliflower.bmp\told72\t1\t1\t2249\t9\t350\r\nold\tmusical_harp_new.bmp\told43\t0\t2\t3063\t9\t350\r\nold\tanimal_farm pig.bmp\told05\t1\t1\t2050\t5\t1375\r\nnew\tanimal_gorilla.bmp\tnew15\t1\t2\t2167\t5\t1375\r\nold\tanimal_chimpanzee.bmp\told03\t1\t1\t3086\t6\t425\r\nold\tkitchen_wooden spoon_old.bmp\told39\t0\t2\t2510\t6\t425\r\nnew\tsporting_windsurf.bmp\tnew31\t1\t2\t2616\t6\t425\r\nold\tkitchen_peppermill.bmp\told33\t1\t1\t2676\t9\t432\r\nold\tanimal_monarch_old.bmp\told08\t0\t2\t1644\t9\t432\r\nnew\tkitchen_baking spatula.bmp\tne

In [592]:
with_encs[3].iloc[2].decoded

'1             Enc           Enc00                       Encoding       0.075         3 \r\n1             Enc           Enc00                       Fixation       3.075         7 \r\n2             Enc           Enc000                      Encoding       10.074        3 \r\n2             Enc           Enc000                      Fixation       13.074        4.5 \r\n3             CTL           CTL0                        Control        17.574        3 \r\n3             CTL           CTL0                        Fixation       20.574        4.5 \r\n4             Enc           Enc01         Old58         Encoding       25.073        3 \r\n4             Enc           Enc01         Old58         Fixation       28.073        0.5 \r\n5             CTL           CTL01                       Control        28.573        3 \r\n5             CTL           CTL01                       Fixation       31.573        0.5 \r\n6             Enc           Enc02         Old02         Encoding       32.073    

In [567]:
with_encs[3].iloc[0].decoded

'*\x00*\x00*\x00 \x00H\x00e\x00a\x00d\x00e\x00r\x00 \x00S\x00t\x00a\x00r\x00t\x00 \x00*\x00*\x00*\x00\r\x00\n\x00V\x00e\x00r\x00s\x00i\x00o\x00n\x00P\x00e\x00r\x00s\x00i\x00s\x00t\x00:\x00 \x001\x00\r\x00\n\x00L\x00e\x00v\x00e\x00l\x00N\x00a\x00m\x00e\x00:\x00 \x00S\x00e\x00s\x00s\x00i\x00o\x00n\x00\r\x00\n\x00L\x00e\x00v\x00e\x00l\x00N\x00a\x00m\x00e\x00:\x00 \x00B\x00l\x00o\x00c\x00k\x00\r\x00\n\x00L\x00e\x00v\x00e\x00l\x00N\x00a\x00m\x00e\x00:\x00 \x00T\x00r\x00i\x00a\x00l\x00\r\x00\n\x00L\x00e\x00v\x00e\x00l\x00N\x00a\x00m\x00e\x00:\x00 \x00S\x00u\x00b\x00T\x00r\x00i\x00a\x00l\x00\r\x00\n\x00L\x00e\x00v\x00e\x00l\x00N\x00a\x00m\x00e\x00:\x00 \x00L\x00o\x00g\x00L\x00e\x00v\x00e\x00l\x005\x00\r\x00\n\x00L\x00e\x00v\x00e\x00l\x00N\x00a\x00m\x00e\x00:\x00 \x00L\x00o\x00g\x00L\x00e\x00v\x00e\x00l\x006\x00\r\x00\n\x00L\x00e\x00v\x00e\x00l\x00N\x00a\x00m\x00e\x00:\x00 \x00L\x00o\x00g\x00L\x00e\x00v\x00e\x00l\x007\x00\r\x00\n\x00L\x00e\x00v\x00e\x00l\x00N\x00a\x00m\x00e\x00:\x00 \x00L\x00o

In [568]:
with_encs[3].iloc[1].decoded
#= with_encs[0]# test = pd.concat(with_encs)
# dir(test['encoding'].unique())

'TrialNumber\tCategory\tTrialCode\tOldNumber\tCorrectSource\tStim_RESP\tStim_RT\tStim_ACC\r\n1\tEnc\tEnc00\t\t5\t\t0\t1\r\n2\tEnc\tEnc000\t\t9\t\t0\t1\r\n3\tCTL\tCTL0\t\t9\t\t0\t1\r\n4\tEnc\tEnc01\tOld58\t8\t\t0\t1\r\n5\tCTL\tCTL01\t\t6\t\t0\t1\r\n6\tEnc\tEnc02\tOld02\t5\t\t0\t1\r\n7\tEnc\tEnc03\tOld54\t5\t\t0\t1\r\n8\tCTL\tCTL02\t\t5\t\t0\t1\r\n9\tCTL\tCTL03\t\t6\t\t0\t1\r\n10\tEnc\tEnc04\tOld69\t6\t\t0\t1\r\n11\tEnc\tEnc05\tOld71\t5\t\t0\t1\r\n12\tCTL\tCTL04\t\t6\t\t0\t1\r\n13\tEnc\tEnc06\tOld48\t9\t\t0\t1\r\n14\tCTL\tCTL05\t\t9\t\t0\t1\r\n15\tEnc\tEnc07\tOld64\t6\t\t0\t1\r\n16\tCTL\tCTL06\t\t9\t\t0\t1\r\n17\tCTL\tCTL07\t\t5\t\t0\t1\r\n18\tEnc\tEnc08\tOld06\t8\t\t0\t1\r\n19\tEnc\tEnc09\tOld75\t9\t\t0\t1\r\n20\tEnc\tEnc10\tOld08\t5\t\t0\t1\r\n21\tEnc\tEnc11\tOld33\t9\t\t0\t1\r\n22\tEnc\tEnc12\tOld37\t5\t\t0\t1\r\n23\tEnc\tEnc13\tOld49\t8\t\t0\t1\r\n24\tEnc\tEnc14\tOld51\t8\t\t0\t1\r\n25\tEnc\tEnc15\tOld42\t9\t\t0\t1\r\n26\tEnc\tEnc16\tOld41\t8\t\t0\t1\r\n27\tEnc\tEnc17\tOld13\t5\t\t0\

In [544]:
with_encs[3].iloc[3].decoded

'*** Header Start ***\r\nVersionPersist: 1\r\nLevelName: Session\r\nLevelName: Block\r\nLevelName: Trial\r\nLevelName: SubTrial\r\nLevelName: LogLevel5\r\nLevelName: LogLevel6\r\nLevelName: LogLevel7\r\nLevelName: LogLevel8\r\nLevelName: LogLevel9\r\nLevelName: LogLevel10\r\nExperiment: Practice\r\nSessionDate: 02-01-2017\r\nSessionTime: 14:10:56\r\nSessionStartDateTimeUtc: 2017-02-01 19:10:56\r\nCodeSubject: 3291977\r\nSubject: 3291977\r\nSession: 1\r\nAssistantResearcherID: 12\r\nDataFile.Basename: Practice-3291977-1\r\nRandomSeed: -1756965767\r\nGroup: 1\r\nDisplay.RefreshRate: 60.021\r\n*** Header End ***\r\n\t\tLevel: 3\r\n\t\t*** LogFrame Start ***\r\n\t\tEncodLevel0: 1\r\n\t\tProcedure: EncodProc0\r\n\t\tCorrectPosition: 4\r\n\t\tPosition: StimRightBottom\r\n\t\tRunning: EncodLevel0\r\n\t\tEncodLevel0.Cycle: 1\r\n\t\tEncodLevel0.Sample: 1\r\n\t\tStim0.OnsetDelay: -1\r\n\t\tStim0.OnsetTime: 165991\r\n\t\tStim0.DurationError: 4\r\n\t\tStim0.RT: 2938\r\n\t\tStim0.RESP: {ENTER}\r\n\

In [545]:
with_encs[3].iloc[4].decoded

'*** Header Start ***\r\nVersionPersist: 1\r\nLevelName: Session\r\nLevelName: Block\r\nLevelName: Trial\r\nLevelName: SubTrial\r\nLevelName: LogLevel5\r\nLevelName: LogLevel6\r\nLevelName: LogLevel7\r\nLevelName: LogLevel8\r\nLevelName: LogLevel9\r\nLevelName: LogLevel10\r\nExperiment: Encoding-scan\r\nSessionDate: 02-01-2017\r\nSessionTime: 15:28:44\r\nSessionStartDateTimeUtc: 2017-02-01 20:28:44\r\nCodeSubject: CIMAQ_3291977\r\nSubject: 3291977\r\nSession: 1\r\nAge: 1\r\nSex: male\r\nScanSession: session2 \r\nTaskVersion: D\r\nMRISite: MNI\r\nResearcherAssistant: 12\r\nTechnologist: Holly\r\nDataFile.Basename: Encoding-scan-3291977-1\r\nRandomSeed: 1779466090\r\nGroup: 1\r\nDisplay.RefreshRate: 60.005\r\n*** Header End ***\r\n\t\tLevel: 3\r\n\t\t*** LogFrame Start ***\r\n\t\tProcedure: EncodProc\r\n\t\tTrialList: 1\r\n\t\tRunning: TrialList\r\n\t\tTrialList.Cycle: 1\r\n\t\tTrialList.Sample: 1\r\n\t\tStim.OnsetDelay: 75\r\n\t\tStim.OnsetTime: 100905\r\n\t\tStim.Duration: 3000\r\n\t\t

In [547]:
with_encs[3].iloc[6].decoded

'%PDF-1.4\n%\n1 0 obj\n<< /Type /Catalog\n/Pages 2 0 R\n>>\nendobj\n\n2 0 obj\n<< /Type /Pages\n/Kids [4 0 R 32 0 R 37 0 R 49 0 R 54 0 R 83 0 R 88 0 R 109 0 R 114 0 R 137 0 R]\n/Count 10\n>>\nendobj\n\n3 0 obj\n<< /ProcSet [/PDF /Text /ImageB /ImageC /ImageI]\n/XObject << /XIPLAYER0 6 0 R\n/XIPLAYER_CM1 8 0 R\n/XIPLAYER_CM2 10 0 R\n/XIPLAYER_CM3 11 0 R\n/XIPLAYER_CM4 12 0 R\n/XIPLAYER_CM5 13 0 R\n/XIPLAYER_CM6 14 0 R\n/XIPLAYER_CM7 15 0 R\n/XIPLAYER_CM8 16 0 R\n/XIPLAYER_CM9 17 0 R\n/XIPLAYER_CM10 18 0 R\n/XIPLAYER_CM11 19 0 R\n/XIPLAYER_CM12 20 0 R\n/XIPLAYER_CM13 21 0 R\n/XIPLAYER_CM14 22 0 R\n/XIPLAYER_CM15 23 0 R\n/XIPLAYER_CM16 24 0 R\n/XIPLAYER_CM17 25 0 R\n/XIPLAYER_CM18 26 0 R\n/XIPLAYER_CM19 27 0 R\n/XIPLAYER_CM20 28 0 R\n/XIPLAYER_CM21 29 0 R\n/XIPLAYER_CM22 30 0 R\n>>\n\n>>\nendobj\n\n4 0 obj\n<< /Type /Page\n/Parent 2 0 R\n/Resources 3 0 R\n/MediaBox [0 0 842 596]\n/Rotate 270\n/Contents 5 0 R\n>>\nendobj\n\n5 0 obj\n<< /Length 656\n/Filter /FlateDecode\n>>\n\nstream\nxV0\x

In [519]:
test = (get_zipcontents(fpath) for fpath in
        loadimages(get_cimaq_dir_paths()[0].zeprimes.fpaths))
next(test)['encoding'] = 

SyntaxError: invalid syntax (<ipython-input-519-d997868aebc6>, line 3)

In [103]:
dir(test.loc[0]['bsheets'])

['__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getnewargs__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mod__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmod__',
 '__rmul__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'capitalize',
 'center',
 'count',
 'decode',
 'endswith',
 'expandtabs',
 'find',
 'fromhex',
 'hex',
 'index',
 'isalnum',
 'isalpha',
 'isascii',
 'isdigit',
 'islower',
 'isspace',
 'istitle',
 'isupper',
 'join',
 'ljust',
 'lower',
 'lstrip',
 'maketrans',
 'partition',
 'replace',
 'rfind',
 'rindex',
 'rjust',
 'rpartition',
 'rsplit',
 'rstrip',
 'split',
 'splitlines',
 'startswith',
 'strip',
 'swapcase',
 'title',
 'translate',
 'upper',
 'zfill']

In [18]:


def get_encoding_zip(fpath):
    contents = get_zipcontents(fpath)
    contents['encoding'] = flatten([[(chardet.detect(zipfile.ZipFile(fpath).open(
                               row[1]['src_names']).read())['encoding'],
                               zipfile.ZipFile(fpath).close())[0]
                           if chardet.detect(zipfile.ZipFile(fpath).open(
                               row[1]['src_names']).read())['encoding'] \
                                   != None else ('cp437', zipfile.ZipFile(fpath).close())[0]]
                               for row in get_zipcontents(fpath).iterrows()])
    return contents

# def get_encoding_zip_singlefunc(fpath):
#     myzip = ZipFile(fpath, 'r')
#     encodings, bsheets, lterms = [], [], []
#     nmlst = getnamelist(fpath)
#     contents = df(tuple(dict(zip(evenodd(itm)[0], evenodd(itm)[1])) for itm
#                         in tuple(tuple(no_ascii(repr(itm)).strip().replace(
#                    "'", "").replace("'",'').replace(
#                        '=',' ')[:-2].split())[1:]
#                    for itm in set(repr(myzip.getinfo(itm)).strip(
#                        ' ').replace(itm, itm.replace(' ', '_')) if ' ' in
#                                   itm else repr(myzip.getinfo(itm)).strip(
#                                       ' ') for itm in nmlst))))
#     contents[['dst_paths', 'exts']] = \
#         [(join(os.path.splitext(fpath)[0], fnm), os.path.splitext(fnm)[1])
#          for fnm in contents.filename]
#     contents[['src_paths', 'src_names']] = \
#         [(join(os.path.splitext(fpath)[0], fnm), fnm) for fnm in nmlst]
#     contents = contents.sort_values('filename').reset_index(drop=True)    
#     for row in tqdm(contents.iterrows()):
#         myfile = myzip.open(row[1].src_names, 'r')
#         encodings.append((row[1]['filename'],
#                           chardet.detect(myfile.read())['encoding']))
#         myfile.close()
#     contents['encoding'] = [itm[1] if itm[1] != None else 'cp437'
#                             for itm in sorted(encodings)]
#     myzip.close()
#     return contents


###############################################################################

# def get_encoding_zip(indir):
#     return list(((get_zipcontents(fpath),
#                   flatten([[(chardet.detect(zipfile.ZipFile(fpath).open(
#                       row[1]['src_names']).read())['encoding'],
#                              zipfile.ZipFile(fpath).close())[0]
#                             if chardet.detect(zipfile.ZipFile(fpath).open(
#                                 row[1]['src_names']).read())['encoding'] \
#                                 != None else ('cp437', zipfile.ZipFile(fpath).close())[0]]
#                                for row in get_zipcontents(fpath).iterrows()])))
#                      for fpath in tqdm(loadimages(indir)))


    
#     contents[0]['encoding'] = flatten(contents[1])
#     myzip = zipfile.ZipFile(contents[0], 'r')
#     contents['encoding'] = [[(chardet.detect(zipfile.ZipFile(fpath).open(
#                                row[1]['src_names']).read())['encoding'],
#                                myfile.close())[0] if chardet.detect(myzip.open(
#                                    row[1]['src_names']).read())['encoding'] != None
#                              else ('cp437')[0]] for row in get_zipcontents(fpath).iterrows()]

 

In [ ]:
with_encs = [get_encoding_zip(fpath) for fpath in
             tqdm(loadimages(get_cimaq_dir_paths()[0].zeprimes.fpaths))]

In [ ]:
with_encs = [get_encoding_zip(fpath) for fpath in
             tqdm(loadimages(get_cimaq_dir_paths()[0].zeprimes.fpaths))]

In [17]:
with_encs = [get_encoding_zip(fpath) for fpath in
             tqdm(loadimages(get_cimaq_dir_paths()[0].zeprimes.fpaths))]


  0%|          | 0/103 [00:00<?, ?it/s]

0it [00:00, ?it/s]

2it [00:01,  1.28it/s]

3it [00:01,  1.69it/s]

4it [00:02,  1.88it/s]

9it [00:02,  3.83it/s]

  1%|          | 1/103 [00:02<04:01,  2.37s/it]

0it [00:00, ?it/s]

4it [00:00,  9.73it/s]

7it [00:00, 11.02it/s]

9it [00:00, 10.58it/s]

  2%|▏         | 2/103 [00:03<03:13,  1.92s/it]

0it [00:00, ?it/s]

6it [00:00, 32.19it/s]

7it [00:00, 10.70it/s]

9it [00:00, 10.69it/s]

  3%|▎         | 3/103 [00:04<02:40,  1.60s/it]

0it [00:00, ?it/s]

1it [00:00,  2.38it/s]

3it [00:02,  1.58it/s]

5it [00:02,  2.18it/s]

10it [00:02,  3.36it/s][A

  4%|▍         | 4/103 [00:07<03:19,  2.02s/it]

3it [00:00, 109.59it/s]A


0it [00:00, ?it/s]

1it [00:01,  1.51s/it]

3it [00:01,  1.08s/it]

4it [00:01,  1.20it/s]

10it [00:02,  4.21it/s][A

  6%|▌         | 6/103 [00:09<02:52,  1.78s/it]

0it [00:00, ?it/s]

1it [00:00,  2.32it/s]

2it [00:00,  2.83it/s]

9it [00:00,  3.86it/s]

10it [00:03,  3.05it/s]

  7%|▋         | 7/103 [00:12<0

 85%|████████▌ | 88/103 [02:33<00:21,  1.44s/it]

0it [00:00, ?it/s]

6it [00:00, 23.11it/s]

7it [00:00, 12.90it/s]

8it [00:03,  1.11it/s]

10it [00:03,  2.89it/s]

 86%|████████▋ | 89/103 [02:37<00:28,  2.05s/it]

0it [00:00, ?it/s]

1it [00:00,  5.81it/s]

5it [00:00,  6.40it/s]

9it [00:00,  9.45it/s]

 87%|████████▋ | 90/103 [02:38<00:22,  1.73s/it]

3it [00:00, 107.86it/s]A


0it [00:00, ?it/s]

1it [00:00,  5.64it/s]

2it [00:00,  4.61it/s]

9it [00:00,  9.31it/s]

 89%|████████▉ | 92/103 [02:39<00:14,  1.36s/it]

0it [00:00, ?it/s]

4it [00:00, 28.76it/s]

8it [00:00, 20.16it/s]

 90%|█████████ | 93/103 [02:39<00:10,  1.08s/it]

0it [00:00, ?it/s]

1it [00:02,  2.30s/it]

7it [00:02,  1.62s/it]

9it [00:02,  3.13it/s]

 91%|█████████▏| 94/103 [02:42<00:14,  1.62s/it]

0it [00:00, ?it/s]

1it [00:00,  2.43it/s]

2it [00:00,  2.71it/s]

5it [00:00,  3.59it/s]

10it [00:03,  2.83it/s][A

 92%|█████████▏| 95/103 [02:46<00:17,  2.20s/it]

0it [00:00, ?it/s]

2it [00:00, 11.50it/s]



100%|██████████| 103/103 [02:07<00:00,  1.24s/it]


In [120]:
with_encs

[                                            filename compress_type  \
 0        3025432_658178_Tache_IRM/3025432_658178.pdf       deflate   
 1  3025432_658178_Tache_IRM/Encoding-scan-3025432...       deflate   
 2  3025432_658178_Tache_IRM/Onset-Event-Encoding_...       deflate   
 3  3025432_658178_Tache_IRM/Output-Responses-Enco...       deflate   
 4  3025432_658178_Tache_IRM/Output_Retrieval_CIMA...       deflate   
 5  3025432_658178_Tache_IRM/PRATIQUE-3025432-1.edat2       deflate   
 6    3025432_658178_Tache_IRM/PRATIQUE-3025432-1.txt       deflate   
 7  3025432_658178_Tache_IRM/Retrieval-3025432-1.e...       deflate   
 8   3025432_658178_Tache_IRM/Retrieval-3025432-1.txt       deflate   
 
      filemode external_attr file_size compress_size  \
 0  -rw-r--r--        0x4000    354022        339482   
 1  -rwxrwxrwx        0x4000     94208         13382   
 2  -rwxrwxrwx        0x4000     21511          3011   
 3  -rwxrwxrwx        0x4000      3557          1206   
 4  -rwx

In [162]:
contents = next((get_zipcontents(fpath) for fpath in
                 loadimages(get_cimaq_dir_paths()[0].zeprimes.fpaths)))

In [160]:
contents

'/home/francois/../../media/francois/seagate_1tb/cimaq_03-19/cimaq_derivatives/task_files/zipped_eprime/3589314_893978_Tache_IRM/3589314_893978_Tache_IRM/Debriefing_1.pdf'

In [240]:
def uzipfiles(indir, outdir=None):
    if outdir != None:
        outdir = outdir
    else:
        outdir = pjoin(dname(indir), 'uz_'+os.path.splitext(bname(indir))[0])
    os.makedirs(outdir, exist_ok=True)
    for item in tqdm([itm for itm in loadimages(indir)
                      if not itm.startswith('.')]):
        with zipfile.ZipFile(item, 'r') as archv:
            archv.extractall(path=pjoin(outdir,
                                       os.path.splitext(bname(item))[0]),
                             members=getnamelist(item))
            archv.close()
    [shutil.move(
        itm,pjoin(outdir,
                  bname(itm).lower().replace(' ', '_')))
     for itm in loadimages(outdir)]
    removeEmptyFolders(indir)
    removeEmptyFolders(outdir)
    return loadfiles(loadimages(outdir))

uzipfiles(get_cimaq_dir_paths()[0].zeprimes.fpaths)

100%|██████████| 103/103 [00:00<00:00, 109.93it/s]


,fname,ext,fpaths
0,.fuse_hidden00000ee000000001,,/home/francois/../../media/francois/seagate_1t...
1,3025432_658178,.pdf,/home/francois/../../media/francois/seagate_1t...
2,3291977_748676,.pdf,/home/francois/../../media/francois/seagate_1t...
3,3400972_956130,.pdf,/home/francois/../../media/francois/seagate_1t...
4,3420680_878354,.pdf,/home/francois/../../media/francois/seagate_1t...
...,...,...,...
1740,output-responses-encoding_cimaq_9003010_session1c,.txt,/home/francois/../../media/francois/seagate_1t...
1741,output-responses-encoding_cimaq_9249304_session1c,.txt,/home/francois/../../media/francois/seagate_1t...
1742,output-responses-encoding_cimaq_9296157_session1a,.txt,/home/francois/../../media/francois/seagate_1t...
1743,output-responses-encoding_cimaq_9494076_sessio...,.txt,/home/francois/../../media/francois/seagate_1t...


In [138]:
def get_dialect(src):
    snffr = csv.Sniffer()
    #.sniff(src.readline())
    src.seek(0)
    lines4test = next(src.splitlines().__iter__)
    return lines4test
#     src.seek(0)
#     for line in lines4test:
#         snffr.sniff(line)
#     valuez = [bname(filename), dialect.delimiter, dialect.doublequote,
#               dialect.escapechar, dialect.lineterminator, dialect.quotechar,
#               dialect.quoting, dialect.skipinitialspace]
#     cnames =['fname', 'delimiter', 'doublequote', 'escapechar',
#              'lineterminator', 'quotechar', 'quoting', 'skipinitialspace']
#     dialect_df = pd.Series(valuez, index=cnames)
#     src.close()
#     return dialect_df

def getnamelist(filename): 
    '''
    Adjustment to ZipFile.nmlst() function to prevent MAC-exclusive
    '__MACOSX' and '.DS_Store' files from interfering.
    Only necessary for files compressed with OS 10.3 or earlier.
    Source: https://superuser.com/questions/104500/what-is-macosx-folder
    Command line solution:
        ``` zip -r dir.zip . -x ".*" -x "__MACOSX"
    Source: https://apple.stackexchange.com/questions/239578/compress-without-ds-store-and-macosx
    '''
    return tuple(sorted(list(itm for itm in
                             zipfile.ZipFile(filename).namelist()
                             if  bname(itm).startswith('.') == False \
                             and '__MACOSX' not in itm \
                             and 'textClipping' not in itm
                             and itm[-1] != '/')))
###############################################################################
def get_zipcontents(fpath):
    myzip = ZipFile(fpath, 'r')
    encodings, bsheets, lterms = [], [], []
    nmlst = getnamelist(fpath)
    contents = df(tuple(dict(zip(evenodd(itm)[0], evenodd(itm)[1])) for itm
                        in tuple(tuple(no_ascii(repr(itm)).strip().replace(
                   "'", "").replace("'",'').replace(
                       '=',' ')[:-2].split())[1:]
                   for itm in set(repr(myzip.getinfo(itm)).strip(
                       ' ').replace(itm, itm.replace(' ', '_')) if ' ' in
                                  itm else repr(myzip.getinfo(itm)).strip(
                                      ' ') for itm in nmlst))))
    contents[['dst_paths', 'exts']] = \
        [(join(os.path.splitext(fpath)[0], fnm), os.path.splitext(fnm)[1])
         for fnm in contents.filename]
    contents[['src_paths', 'src_names']] = \
        [(join(os.path.splitext(fpath)[0], fnm), fnm) for fnm in nmlst]
    contents = contents.sort_values('filename')
    for row in tqdm(contents.iterrows()):
        myfile = myzip.open(row[1].src_names, 'r')
        encodings.append((row[1]['filename'],
                          chardet.detect(myfile.read())['encoding']))
        myfile.close()
    contents['encoding'] = [itm[1] if itm[1] != None else 'cp437'
                            for itm in sorted(encodings)] 
    myzip.close()
    return contents
    
# def get_zipinfos(fpath):
# #     with ZipFile(fpath, 'r') as myzip:
#     encodings, bsheets, lterms = [], [], []
#     namelist = getnamelist(fpath)
#     contents = get_zipcontents(fpath)
#     for row in tqdm(contents.iterrows()):
#         with zipfile.ZipFile(row[1].src_paths, 'r') as myzip:
#             encodings.append(row[1]['filename'], chardet.detect(myzip.read())['encoding', 'confidence'])
#         myzip.close()
#     contents['encoding'] = [itm[1] for itm in sorted(encodings)]
#     return contents


In [132]:
zipfiles = pd.concat([get_zipcontents(fname) for fname in
                      loadimages(get_cimaq_dir_paths()[0].zeprimes.fpaths)],
                     ignore_index=True, verify_integrity=True).sort_values('filename').reset_index(drop=True)

9it [00:01,  4.67it/s]
9it [00:00, 22.59it/s]
9it [00:00, 22.42it/s]
10it [00:02,  4.04it/s]
3it [00:00, 262.63it/s]
10it [00:01,  5.31it/s]
10it [00:02,  3.74it/s]
9it [00:00, 22.03it/s]
9it [00:00, 21.88it/s]
9it [00:00, 21.78it/s]
13it [00:00, 13.27it/s]
10it [00:01,  7.11it/s]
9it [00:00, 30.66it/s]
9it [00:00, 22.08it/s]
10it [00:01,  7.95it/s]
9it [00:00, 21.73it/s]
13it [00:01, 11.87it/s]
9it [00:00, 22.02it/s]
9it [00:00, 22.45it/s]
9it [00:00, 22.04it/s]
10it [00:01,  7.44it/s]
13it [00:00, 14.28it/s]
10it [00:07,  1.30it/s]
10it [00:03,  3.21it/s]
9it [00:00, 22.29it/s]
10it [00:01,  7.35it/s]
9it [00:00, 22.30it/s]
9it [00:00, 22.03it/s]
10it [00:02,  3.72it/s]
9it [00:02,  4.25it/s]
10it [00:01,  6.35it/s]
8it [00:00, 20.94it/s]
10it [00:02,  3.96it/s]
10it [00:02,  4.02it/s]
10it [00:02,  4.20it/s]
10it [00:02,  4.24it/s]
9it [00:00, 20.71it/s]
10it [00:02,  3.80it/s]
9it [00:00, 21.96it/s]
10it [00:01,  7.21it/s]
9it [00:00, 23.83it/s]
9it [00:00, 24.16it/s]
9it [00:00, 2

In [134]:
zipfiles['encoding'].unique()

array(['ascii', 'KOI8-R', 'UTF-16', 'ISO-8859-1', 'cp437', 'MacCyrillic',
       'Windows-1252', 'windows-1253', 'Windows-1254', 'Windows-1253',
       'IBM866'], dtype=object)

In [51]:
def get_zipinfos(fpath):
#     with ZipFile(fpath, 'r') as myzip:
    encodings, bsheets, lterms = [], [], []
    namelist = getnamelist(fpath)
    contents = get_zipcontents(fpath)
    for row in tqdm(contents.iterrows()):
        with zipfile.ZipFile(row[1].src_paths, 'r') as myzip:
            encodings.append(row[1]['filename'], chardet.detect(myzip.read())['encoding', 'confidence'])
        myzip.close()
    contents['encoding'] = [itm[1] for itm in sorted(encodings)]
    return contents
#             bsheets.append((row[1]['filename'], myfile.read()))
# #             '''https://www.prepressure.com/pdf/basics/fileformat#:~:text=PDF%20files%20are%20either%208,to%20create%20the%20PDF%20file).'''
# #             with myzip.open(row[1]['src_names']) as myfile:
#             myfile.seek(0)                   
#             encod = chardet.detect((myfile.read()))['encoding']
#             if encod == None:
#                 encod = 'utf16'
#             myfile.seek(0)                
#             encodings.append((row[1]['filename'], encod))
#             myfile.close()
        myzip.close()
        contents['encoding'] = [itm[1] for itm in sorted(encodings)]
#         contents['bytes_sheets'] = [itm[1] for itm in sorted(bsheets)]
#         contents['lineterminator'] = [[line for line in
#                                        row[1]['bytes_sheets'].splitlines(
#                                            keepends=True)][0].strip([line for line in
#                                              row[1]['bytes_sheets'].splitlines()][0])
#                                       for row in contents.iterrows()]
# #         contents['delimiter'] = [[csv.Sniffer().sniff(next((line for line in row[1]['bytes_sheets'].splitlines())).decode(row[1]['encoding'])).delimiter]
# #                                  for row in contents.iterrows()]
#         contents['n_lines'] = [len(row[1]['bytes_sheets'].splitlines())
#                                for row in contents.iterrows()]
#         contents['bytes_sheets'] = [itm[1] for itm in sorted(bsheets)]
    return contents
arch_contents = get_zipinfos

In [67]:
dlst, patterns = get_cimaq_dir_paths()
zfiles = pd.concat(next(get_zipinfos(next(loadimages(dlst.zeprimes.fpaths).__iter__())).iterrows()).__iter__())
zfiles

TypeError: cannot concatenate object of type '<class 'int'>'; only Series and DataFrame objs are valid

In [43]:
# allinfos = pd.concat([get_zipinfos(fpath) for fpath in zfiles.fpaths],
#                      axis=1, join='outer', ignore_index=True)
allinfos = [get_zipinfos(fpath) for fpath in zfiles.fpaths]


9it [00:01,  4.62it/s]
9it [00:00, 22.76it/s]
9it [00:00, 23.34it/s]
10it [00:02,  4.17it/s]
3it [00:00, 258.84it/s]
10it [00:01,  5.55it/s]
10it [00:02,  3.87it/s]
9it [00:00, 22.72it/s]
9it [00:00, 22.48it/s]
9it [00:00, 22.47it/s]
13it [00:00, 13.61it/s]
10it [00:01,  7.18it/s]
9it [00:00, 30.14it/s]
9it [00:00, 21.85it/s]
10it [00:01,  8.23it/s]
9it [00:00, 23.88it/s]
13it [00:01, 12.53it/s]
9it [00:00, 24.40it/s]
9it [00:00, 24.72it/s]
9it [00:00, 24.22it/s]
10it [00:01,  8.20it/s]
13it [00:00, 15.72it/s]
10it [00:07,  1.41it/s]
10it [00:02,  3.47it/s]
9it [00:00, 24.24it/s]
10it [00:01,  7.83it/s]
9it [00:00, 24.13it/s]
9it [00:00, 23.89it/s]
10it [00:02,  4.04it/s]
9it [00:02,  4.20it/s]
10it [00:01,  5.89it/s]
8it [00:00, 19.87it/s]
10it [00:02,  3.84it/s]
10it [00:02,  3.94it/s]
10it [00:02,  4.06it/s]
10it [00:02,  4.31it/s]
9it [00:00, 22.20it/s]
10it [00:02,  3.74it/s]
9it [00:00, 22.01it/s]
10it [00:01,  6.53it/s]
9it [00:00, 21.82it/s]
9it [00:00, 22.09it/s]
9it [00:00, 2

In [47]:
allinfos[10]

,filename,compress_type,filemode,external_attr,file_size,compress_size,fpaths,src_names,exts,encoding,bytes_sheets,lineterminator,n_lines
0,3589314_893978_Tache_IRM/Debriefing_1.pdf,deflate,-rwxr-xr-x,0x4000,39105,25527,/home/francois/../../media/francois/seagate_1t...,3589314_893978_Tache_IRM/Debriefing 1.pdf,.pdf,Windows-1254,b'%PDF-1.4\n1 0 obj\n<</Type /XObject /Subtype...,b'\n',1122
1,3589314_893978_Tache_IRM/Debriefing_2.pdf,deflate,-rwxr-xr-x,0x4000,39649,32213,/home/francois/../../media/francois/seagate_1t...,3589314_893978_Tache_IRM/Debriefing 2.pdf,.pdf,utf16,b'%PDF-1.4\n1 0 obj\n<</Type /XObject /Subtype...,b'\n',869
2,3589314_893978_Tache_IRM/Encoding-scan-3589314...,deflate,-rwxr-xr-x,0x4000,94720,13589,/home/francois/../../media/francois/seagate_1t...,3589314_893978_Tache_IRM/Encoding-scan-3589314...,.edat2,Windows-1254,b'\xd0\xcf\x11\xe0\xa1\xb1\x1a\xe1\x00\x00\x00...,b'\n',388
3,3589314_893978_Tache_IRM/Encoding-scan-3589314...,deflate,-rwxr-xr-x,0x4000,206476,12751,/home/francois/../../media/francois/seagate_1t...,3589314_893978_Tache_IRM/Encoding-scan-3589314...,.txt,UTF-16,b'\xff\xfe*\x00*\x00*\x00 \x00H\x00e\x00a\x00d...,b'\r',9625
4,3589314_893978_Tache_IRM/Fiche_de_Suivi.pdf,deflate,-rwxr-xr-x,0x4000,28370,21577,/home/francois/../../media/francois/seagate_1t...,3589314_893978_Tache_IRM/Fiche de Suivi.pdf,.pdf,utf16,b'%PDF-1.4\n1 0 obj\n<</Type /XObject /Subtype...,b'\n',652
5,3589314_893978_Tache_IRM/Onset-Event-Encoding_...,deflate,-rwxr-xr-x,0x4000,21512,2826,/home/francois/../../media/francois/seagate_1t...,3589314_893978_Tache_IRM/Onset-Event-Encoding_...,.txt,ascii,b'1 Enc Enc00 ...,b'\r\n',240
6,3589314_893978_Tache_IRM/Output-Responses-Enco...,deflate,-rwxr-xr-x,0x4000,3533,1190,/home/francois/../../media/francois/seagate_1t...,3589314_893978_Tache_IRM/Output-Responses-Enco...,.txt,ascii,b'TrialNumber\tCategory\tTrialCode\tOldNumber\...,b'\r\n',121
7,3589314_893978_Tache_IRM/Output_Retrieval_CIMA...,deflate,-rwxr-xr-x,0x4000,5875,2213,/home/francois/../../media/francois/seagate_1t...,3589314_893978_Tache_IRM/Output_Retrieval_CIMA...,.txt,ISO-8859-1,b'category\tStim\tOldNumber\tRecognition_ACC\t...,b'\r\n',118
8,3589314_893978_Tache_IRM/PRATIQUE-3589314-1.edat2,deflate,-rwxr-xr-x,0x4000,41472,5950,/home/francois/../../media/francois/seagate_1t...,3589314_893978_Tache_IRM/PRATIQUE-3589314-1.edat2,.edat2,Windows-1254,b'\xd0\xcf\x11\xe0\xa1\xb1\x1a\xe1\x00\x00\x00...,b'\n',243
9,3589314_893978_Tache_IRM/PRATIQUE-3589314-1.txt,deflate,-rwxr-xr-x,0x4000,67960,4472,/home/francois/../../media/francois/seagate_1t...,3589314_893978_Tache_IRM/PRATIQUE-3589314-1.txt,.txt,UTF-16,b'\xff\xfe*\x00*\x00*\x00 \x00H\x00e\x00a\x00d...,b'\r',2695


In [49]:
df(tuple(evenodd(tuple(range(0, 16)))))

,0,1,2,3,4,5,6,7
0,0,2,4,6,8,10,12,14
1,1,3,5,7,9,11,13,15


In [50]:
archv_path = zfiles.loc[0].fpaths
archv_path

'/home/francois/../../media/francois/seagate_1tb/cimaq_03-19/cimaq_derivatives/task_files/zipped_eprime/3025432_658178_Tache_IRM.zip'

In [676]:

        contents[['pscid', 'dccid']] = \
                [(next((itm for itm in ar_name.split('_') if len(itm) \
                        == 7 and itm.isnumeric())),
                  next((itm for itm in ar_name.split('_')
                        if len(itm) == 6 and itm.isnumeric())))
                 for ar_name in contents.archv_name]
        for row in contents.iterrows():
            with myzip.open(row[1]['filename']) as myfile:
                rawsheets.append((row[1]['filename'], myfile.read()))
            myfile.close()
    myzip.close()
    contents = contents.sort_values('filename')
    contents['rawsheets'] = [rsheet[1] for rsheet in sorted(rawsheets)]
    contents['encodings'] = [chardet.detect(rsheet[1])['encoding']
                             for rsheet in sorted(rawsheets)]
    contents['hdr'] = [row[1]['rawsheets'].splitlines(keepends=False)[0][0] not
                       in b'.-0123456789' for row in contents.iterrows()]
    contents['decoded'] = [no_ascii(row[1]['rawsheets'].decode(row[1]['encodings']).lower())
                              if row[1]['encodings'] != None else
                              repr(row[1]['rawsheets']).lower()
                              for row in contents.iterrows()]
    
    contents['lineterminator'] = [row[1]['decoded'].splitlines(keepends=True)[0].strip(
                                     row[1]['decoded'].splitlines(keepends=False)[0])
                                  for row in contents.iterrows()]
    contents['delimiter'] = [csv.Sniffer().sniff(row[1]['decoded'].splitlines()[0]).delimiter
                             for row in contents.iterrows()]
    contents['repr_decoded'] = [str(row[1]['decoded']) for row in contents.iterrows()]
    
    contents['split_delim'] = [tuple(line.strip().replace(row[1]['delimiter'], '').replace(' ', '_').split(row[1]['delimiter'])+['\n']
#                                      if row[1]['delimiter']
                                     for line in row[1]['decoded'].splitlines(keepends=False))
                               for row in contents.iterrows()]    
    contents[['nrows', 'colsperrows']] = [(len(row[1]['rawsheets'].splitlines()),
                                                   tuple(len(line.split()) for line
                                                         in row[1]['decoded'].splitlines()))
                                          for row in contents.iterrows()]
    contents['dupindex'] = [evenodd(row[1]['decoded'].splitlines())[0][0] == \
                            evenodd(row[1]['decoded'].splitlines())[0][0]
                            for row in contents.iterrows()]
    
    contents['bytes_width'] = [len(line) for line in row[1]['rawsheets'].splitlines()
                               for line in row[1]['rawsheets'].splitlines()[1:]]
#     contents['dupindex'] = [evrows == odrows for (evrows, odrows) in
#                             evenodd(df(row[1]['split_delim']).index)
#                             for row in contents.iterrows()]    
#     contents['dupindex'] = [[(itm[0][0] == itm[1][0]) for itm in evenodd(row[1]['decoded'].splitlines())]
#                             for row in _.iterrows()]
    
#     contents['repr_noend_decoded'] = [line.join('\n') for line in repr(row[1]['decoded'].splitlines(keepends=False))
#                                       for row in contents.iterrows()]

    return contents.sort_values('filename').reset_index(drop=True)

In [677]:
all_bytes_sheets = get_zipinfos(archv_path)
all_bytes_sheets

/home/francois/.local/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


KeyError: 'rawsheets'

In [ ]:
# from decimal import Decimal
# from tabulate import tabulate

def replace_all(text, dic):
    for i, j in dic.iteritems():
        text = text.replace(i, j)
    return text
'_______________________'
delimsheets = all_bytes_sheets.loc[[2, 3, 4]]['split_delim']
decodsheets = all_bytes_sheets.loc[[2, 3, 4]]['decoded']
# flatten([line+'\n'.split() for line in decodsheets[2].splitlines()])
tuple(line.strip().split() for line in decodsheets[2].splitlines())
# evenodd(delimsheets[2])[0][0], evenodd(delimsheets[2])[1][0][0]
# tuple(str(round(float(itm), 1)) if itm.isnumeric() else itm for itm in delimsheets[4][44] )

In [ ]:
# df((itm for itm in delimsheets[2]))
# repr(json.dumps())
delimsheets[3]

In [ ]:
delimsheets[4]

In [ ]:
tabulate(delimsheets[3])
delimsheets[3]

In [668]:
def mksheet(inpt):
    evrows, odrows = evenodd(tuple(tuple(line.split()) for line in
                               inpt.splitlines()))
    return tuple(tuple(itm[0]+itm[1]) for itm in tuple(zip(evrows, odrows)))

testsheets = [mksheet(sheet) for sheet in 
# tuple('\t'.join("{:.2f}".format(float(itm)) if itm.isnumeric() else itm for itm in line)
#        for line in testsheets[2])
# dict(csv.Sniffer().sniff((testsheets[2]).splitlines()[0]))
# str.center(json.dumps(tuple(dict(testsheets[2]).items())))

SyntaxError: unexpected EOF while parsing (<ipython-input-668-26da57c16088>, line 10)

In [ ]:
help(float)

In [ ]:
testsheets[1] 

In [ ]:
testsheets[2]

In [ ]:


# display([no_ascii(line.strip(all_bytes_sheets.loc[2].delimiter)) for line in
#                     all_bytes_sheets.loc[2].repr_decoded.splitlines()])
# #         all_bytes_sheets.loc[3].repr_decoded,
#         all_bytes_sheets.loc[4].repr_decoded)

display(all_bytes_sheets.loc[6].repr_decoded)
#         all_bytes_sheets.loc[2].repr_noend_decoded
        #.rpartition(all_bytes_sheets.loc[2].delimiter),
        #.partition(all_bytes_sheets.loc[2].delimiter))
# display(all_bytes_sheets.iloc[8]['split_bytes_w_ends'][0],
# bytes(all_bytes_sheets.iloc[8]['split_bytes_w_ends'][0]))
#[-1]).decode('utf8')
# bytes(all_bytes_sheets.iloc[8]['split_bytes_n_ends'][0][-1])
#[['encodings', 'encodings2']]

In [ ]:
help(json.dumps)

In [ ]:
pd.Series([line.strip(bytes(itm for itm in line.strip()))
           for line in all_bytes_sheets.loc[3].rawsheets.splitlines()]).unique()[0]

In [ ]:
evesheet, oddsheet = evenodd(all_bytes_sheets.loc[0]['splitsheets'])
1#display(evesheet[0].split()
display(oddsheet[0].split(), oddsheet[0].split(all_bytes_sheets.loc[0]['delimiter']),
       '\t'.join(oddsheet[0].split(all_bytes_sheets.loc[0]['delimiter'])))

In [ ]:
all_bytes_sheets.loc[0]['splitlines_bytes'] = [line for line in all_bytes_sheets.loc[0]['rawsheets']]

In [ ]:
a_test = df(all_bytes_sheets.loc[0]['decodsheets'].split(all_bytes_sheets.loc[0]['lineterminator']))
b_test = df(all_bytes_sheets.loc[0]['decodsheets'].splitlines())

a_test, b_test

In [ ]:
pd.Series(all_bytes_sheets.loc[3]['splitsheets'][0].split(all_bytes_sheets.loc[3]['delimiter'])).unique()
#, all_bytes_sheets.loc[0]['splitsheets'], all_bytes_sheets.loc[0]['splitsheets']

In [ ]:
( all_bytes_sheets['fpaths'].values)#[0].replace('category', ' ').replace('stim', ' ')

In [456]:
tuple([itm[0].replace(subitm, '') for subitm in itm[1]] for itm in all_bytes_sheets.loc[0]['test_string'])

KeyError: 'test_string'

In [443]:
help(bytes(b'\xe7').title())

Help on bytes object:

class bytes(object)
 |  bytes(iterable_of_ints) -> bytes
 |  bytes(string, encoding[, errors]) -> bytes
 |  bytes(bytes_or_buffer) -> immutable copy of bytes_or_buffer
 |  bytes(int) -> bytes object of size given by the parameter initialized with null bytes
 |  bytes() -> empty bytes object
 |  
 |  Construct an immutable array of bytes from:
 |    - an iterable yielding integers in range(256)
 |    - a text string encoded using the specified encoding
 |    - any object implementing the buffer API.
 |    - an integer
 |  
 |  Methods defined here:
 |  
 |  __add__(self, value, /)
 |      Return self+value.
 |  
 |  __contains__(self, key, /)
 |      Return key in self.
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __getitem__(self, key, /)
 |      Return self[key].
 |  
 |  __getnewargs__(...)
 |  
 |  __g

In [444]:
len(all_bytes_sheets.loc[2]['splitsheets'])

1

In [479]:
all_bytes_sheets.iloc[4]['splitsheets']


['category\tstim\toldnumber\trecognition_acc\trecognition_resp\trecognition_rt\tspatial_resp\tspatial_rt\tspatial_acc(à corriger voir output-encodage)',
 'old\tanimal_penguin.bmp\told10\t1\t1\t3216\t8\t462',
 'old\tsporting_bicycle_old.bmp\told55\t0\t2\t9409\t8\t462',
 'old\tfruit_lemon_new.bmp\told69\t1\t1\t2746\t5\t4435',
 'new\tvegie_broccoli.bmp\tnew36\t1\t2\t1633\t5\t4435',
 'old\tanimal_zebra_old.bmp\told13\t1\t1\t1311\t8\t369',
 'old\tkitchen_apron.bmp\told27\t1\t1\t1105\t8\t379',
 'old\tanimal_monarch_old.bmp\told08\t1\t1\t1763\t5\t436',
 'new\tkitchen_toaster_old.bmp\tnew30\t1\t2\t1257\t5\t436',
 'new\tmusical_uculele_new.bmp\tnew39\t0\t1\t4342\t8\t455',
 'new\tfruit_cherry.bmp\told67\t1\t1\t1611\t6\t2122',
 'new\tfood_walnut01c.bmp\tnew34\t1\t2\t1348\t6\t2122',
 'old\tfood_pickle01a.bmp\told22\t1\t1\t1029\t8\t371',
 'new\tvegie_pumpkin.bmp\tnew21\t1\t2\t1795\t8\t371',
 'old\tanimal_lion_new.bmp\told07\t1\t1\t12322\t8\t488',
 'new\tsporting_flippers.bmp\tnew02\t1\t2\t1267\t8\t

In [336]:
[no_ascii(line).encode().strip().replace(b'\'', b'\s').strip().split() for line in all_bytes_sheets['decodsheets'].loc[7].split()]

[[b'\\sb\\\\s%PDF-1.4\\\\n%\\\\xf8\\\\xe5\\\\xd0\\\\xc4\\\\xc6\\\\n1'],
 [b'0'],
 [b'obj\\\\n<<'],
 [b'/Type'],
 [b'/Catalog\\\\n/Pages'],
 [b'2'],
 [b'0'],
 [b'R\\\\n>>\\\\nendobj\\\\n\\\\n2'],
 [b'0'],
 [b'obj\\\\n<<'],
 [b'/Type'],
 [b'/Pages\\\\n/Kids'],
 [b'[4'],
 [b'0'],
 [b'R'],
 [b'16'],
 [b'0'],
 [b'R'],
 [b'23'],
 [b'0'],
 [b'R'],
 [b'42'],
 [b'0'],
 [b'R'],
 [b'54'],
 [b'0'],
 [b'R'],
 [b'79'],
 [b'0'],
 [b'R]\\\\n/Count'],
 [b'6\\\\n>>\\\\nendobj\\\\n\\\\n3'],
 [b'0'],
 [b'obj\\\\n<<'],
 [b'/ProcSet'],
 [b'[/PDF'],
 [b'/Text'],
 [b'/ImageB'],
 [b'/ImageC'],
 [b'/ImageI]\\\\n/XObject'],
 [b'<<'],
 [b'/XIPLAYER0'],
 [b'6'],
 [b'0'],
 [b'R\\\\n/XIPLAYER_CM1'],
 [b'8'],
 [b'0'],
 [b'R\\\\n/XIPLAYER_CM2'],
 [b'10'],
 [b'0'],
 [b'R\\\\n/XIPLAYER_CM3'],
 [b'11'],
 [b'0'],
 [b'R\\\\n/XIPLAYER_CM4'],
 [b'12'],
 [b'0'],
 [b'R\\\\n/XIPLAYER_CM5'],
 [b'13'],
 [b'0'],
 [b'R\\\\n/XIPLAYER_CM6'],
 [b'14'],
 [b'0'],
 [b'R\\\\n>>\\\\n\\\\n>>\\\\nendobj\\\\n\\\\n4'],
 [b'0'],
 [b'obj\\\\n<<'

In [203]:
all_bytes_sheets['decodsheets'].loc[5]

'TrialNumber\tCategory\tTrialCode\tOldNumber\tCorrectSource\tStim_RESP\tStim_RT\tStim_ACC\r\n1\tCTL\tCTL0\t\t5\t1\t1183\t0\r\n2\tEnc\tEnc00\t\t8\t1\t2759\t0\r\n3\tEnc\tEnc000\t\t9\t1\t1656\t0\r\n4\tCTL\tCTL01\t\t8\t1\t1402\t0\r\n5\tEnc\tEnc01\tOld60\t6\t\t0\t1\r\n6\tEnc\tEnc02\tOld01\t5\t1\t1822\t0\r\n7\tEnc\tEnc03\tOld24\t6\t1\t1930\t0\r\n8\tEnc\tEnc04\tOld68\t9\t1\t1814\t0\r\n9\tEnc\tEnc05\tOld55\t8\t1\t1094\t0\r\n10\tEnc\tEnc06\tOld46\t9\t1\t2895\t0\r\n11\tCTL\tCTL02\t\t5\t1\t838\t0\r\n12\tCTL\tCTL03\t\t6\t1\t807\t0\r\n13\tEnc\tEnc07\tOld38\t8\t1\t2139\t0\r\n14\tCTL\tCTL04\t\t8\t1\t791\t0\r\n15\tEnc\tEnc08\tOld47\t6\t1\t1678\t0\r\n16\tEnc\tEnc09\tOld20\t5\t1\t2383\t0\r\n17\tCTL\tCTL05\t\t8\t1\t840\t0\r\n18\tEnc\tEnc10\tOld30\t6\t1\t2578\t0\r\n19\tEnc\tEnc11\tOld05\t6\t1\t1558\t0\r\n20\tEnc\tEnc12\tOld25\t8\t1\t2848\t0\r\n21\tEnc\tEnc13\tOld77\t5\t1\t2340\t0\r\n22\tEnc\tEnc14\tOld17\t9\t1\t2740\t0\r\n23\tEnc\tEnc15\tOld35\t9\t1\t2095\t0\r\n24\tEnc\tEnc16\tOld32\t5\t1\t1966\t0\r\n25\t

In [119]:
display(chardet.detect(all_bytes_sheets.iloc[3]['rawsheets']))

{'encoding': 'ascii', 'confidence': 1.0, 'language': ''}

In [225]:
dlst, patterns = get_cimaq_dir_paths()
uzipfiles(indir = dlst.zeprimes.fpaths,
          outdir = dlst.uzeprimes.fpaths)
cimaq_filter()
prefixes = pd.Series(('_'.join(val.split('_')[:2])
                      for val in loadfiles(loadimages(dlst.zeprimes.fpaths)).fname)).unique()
indiv_patterns = tuple(item for item
                           in dict(zip(prefixes, prefixes)).items())
prefixes, indiv_patterns
#     {"indiv_patterns": tuple(item for item\\
#                        in dict(zip(prefixes, prefixes)).items()),
#      "prefx": pd.Series(('_'.join(val.split('_')[:2]) for val in
#                          loadfiles(loadimages(dlst['uzeprimes'][0])).fname)).unique()}

KeyError: 'cimaq_dir'

(array([], dtype=float64), ())

In [2]:

def fix_cimaq(cimaq_dir):
    # Paths to files and direcotries
    cimaq_dir = xpu(cimaq_dir)
    dlst, prefixes = get_cimaq_dir_paths(cimaq_dir)

    # Name and indexing patterns and prefixes
#     mean_sheets_patterns = (('dccid', '(?<!\d)\d{6}(?!\d)'),
#                             ('pscid', '(?<!\d)\d{7}(?!\d)'))
    prefixes = pd.Series(('_'.join(val.split('_')[:2])
                          for val in loadfiles(dlst['uzeprimes'][0]).fname)).unique()
    indiv_patterns = tuple(item for item
                               in dict(zip(prefixes, prefixes)).items())
    
    # Structure file paths and participant identifiers in a DataFrame
    events_sheets = loadfiles(indir=dlst['uzeprimes'][0])
    confounds_sheets = loadfiles(dlst['confdir'][0])
    events_sheets['pscid'] = [re.compile('(?<!\d)\d{7}(?!\d)').search(fname).group()
                       for fname in events_sheets.fname]
    
    confounds_sheets['dccid'] = [re.compile('(?<!\d)\d{6}(?!\d)').search(fname).group()
                       for fname in loadfiles(dlst['confdir'][0]).fname]
    
    # Get parsing, dialect and encoding information with 'get_infos' function
    events_infos = df((get_infos(fpath) for fpath in events_sheets.fpaths))
    confounds_infos = df((get_infos(fpath) for fpath in confounds_sheets.fpaths))
    events_infos = sortmap(pd.merge(events_infos, events_sheets,
                                    how='outer'), indiv_patterns)
    confounds_infos = pd.merge(confounds_infos, confounds_sheets, how='outer')
    mean_parsing_infos = df((get_infos(fpath) for fpath in
                             loadfiles(pathlist = dlst['mean_paths']).fpaths))
    
    # Group information about participants who passed QC assessment
    new_mean_sheets = OrderedDict((row[1]['fpaths'],
                                   prep_sheet(filename = row[1]['fpaths'],
                                encoding=row[1]['encoding'],
                                hdr = row[1]['has_header'],
                                n_fields = row[1]['n_fields'],
                                delimiter = row[1]['delimiter'],
                                dupindex = row[1]['dup_index'],
                                row_breaks = row[1]['row_breaks'],
                                r_rowbreaks = row[1]['r_rowbreaks'],
                                n_lines = row[1]['n_lines'],
                                width = row[1]['width'],
                                lineterminator = row[1]['lineterminator']))
                     for row in mean_parsing_infos.iterrows())
    new_mean_sheets = dict(zip(new_mean_sheets.keys(),
                               tuple(sheet.convert_dtypes(str)
                                     for sheet in rename_imposter_cols(
                   [sheet for sheet in new_mean_sheets.values()],
                   mean_sheets_patterns))))
    min_ind = df([sheet.dccid for sheet in new_mean_sheets.values()
               if sheet.shape[0] == pd.Series(
                   [sheet.shape[0] for sheet in
                    new_mean_sheets.values()]).min()]).T
    inds = [sheet[1].dccid.tolist() for sheet in new_mean_sheets.items()]
    new_mean_sheets = dict((item[0], item[1].set_index(
                          'dccid').loc[min_ind.dccid].reset_index(drop=False))
                           for item in new_mean_sheets.items())
    renamer = [sheet[['dccid', 'pscid']] for sheet in new_mean_sheets.values()
               if 'dccid' and 'pscid' in sheet.columns][0].astype(str).dropna()
    events_infos = pd.merge(events_infos, renamer, on='pscid', how='outer').dropna(how='any')
    confounds_infos = pd.merge(confounds_infos, renamer, on='dccid', how='outer').dropna(how='any')
    all_infos = pd.concat([events_infos, confounds_infos]).fillna(False).reset_index(drop=True)
    all_infos['confounds'] = ['confounds' in fname for fname in all_infos.fname]
    all_infos['prefix'] = [find_key(row[1][['output_responses', 'onset_event',
                               'output_retrieval', 'confounds']].T.to_dict(), True)
                           for row in all_infos.iterrows()]
    all_infos.to_csv(join(cimaq_dir, 'all_parsing_infos.csv'), index=False)

    # Use 'prep_sheet' function to fix errors by reading data as a stream of bytes
    new_events = tuple((''.join(['sub-', row[1]['pscid'], '-', row[1]['dccid'],
                                '_', row[1]['prefix'], '.csv']),
                        prep_sheet(filename = row[1]['fpaths'],
                                   encoding=row[1]['encoding'],
                                   hdr = row[1]['has_header'],
                                   n_fields = row[1]['n_fields'],
                                   delimiter = row[1]['delimiter'],
                                   dupindex = row[1]['dup_index'],
                                   row_breaks = row[1]['row_breaks'],
                                   r_rowbreaks = row[1]['r_rowbreaks'],
                                   n_lines = row[1]['n_lines'],
                                   width = row[1]['width'],
                                   lineterminator = row[1]['lineterminator']))
                       for row in tqdm(all_infos.iterrows()))
    newdir = join(cimaq_dir, 'cleaned_events_and_confounds')
    os.makedirs(newdir, exist_ok=True)
    next((itm[1].to_csv(join(newdir, itm[0]), index=False)
          for itm in new_events))
    
def main():
    fix_cimaq(cimaq_dir = '~/../../media/francois/seagate_1tb/cimaq_03-19/cimaq_derivatives')

if __name__ == "__main__":
    main()


373it [01:38,  3.78it/s]


In [194]:
# fp = xpu('~/cimaq_memory/cimaq_dir_list.json')
cimaq_dir = xpu('~/../../media/francois/seagate_1tb/cimaq_03-19/cimaq_derivatives')


{'mean_paths': ['/home/francois/../../media/francois/seagate_1tb/cimaq_03-19/cimaq_derivatives/participants/MotionResults/fMRI_meanMotion.tsv',
  '/home/francois/../../media/francois/seagate_1tb/cimaq_03-19/cimaq_derivatives/participants/MemoTaskParticipantFile.tsv',
  '/home/francois/../../media/francois/seagate_1tb/cimaq_03-19/cimaq_derivatives/participants/Neuropsych/ALL_Neuropsych_scores.tsv',
  '/home/francois/../../media/francois/seagate_1tb/cimaq_03-19/cimaq_derivatives/participants/Participants_bids.tsv',
  '/home/francois/../../media/francois/seagate_1tb/cimaq_03-19/cimaq_derivatives/participants/sub_list_TaskQC.tsv',
  '/home/francois/../../media/francois/seagate_1tb/cimaq_03-19/cimaq_derivatives/participants/TaskResults/fMRI_behavMemoScores.tsv'],
 'new_mean_dir': ['/home/francois/../../media/francois/seagate_1tb/cimaq_03-19/cimaq_derivatives/mean_sheets'],
 'confdir': ['/home/francois/../../media/francois/seagate_1tb/cimaq_03-19/cimaq_derivatives/confounds'],
 'uzeprimes': 

In [190]:
read2 = json_read(fp2, 'rb')
read2

{'confdir': ['confounds'],
 'mean_paths': ['participants/MotionResults/fMRI_meanMotion.tsv',
  'participants/MemoTaskParticipantFile.tsv',
  'participants/Neuropsych/ALL_Neuropsych_scores.tsv',
  'participants/Participants_bids.tsv',
  'participants/sub_list_TaskQC.tsv',
  'participants/TaskResults/fMRI_behavMemoScores.tsv'],
 'new_mean_dir': ['mean_sheets'],
 'uzeprimes': ['task_files/uzeprimes']}

In [37]:
def _update_timestamp(path: os.PathLike, set_new: bool) -> None:
    """
    Context manager to set the timestamp of the path to plus or
    minus a fixed delta, regardless of modifications within the context.

    if set_new is True, the delta is added. Otherwise, the delta is subtracted.
    """
    stats = os.stat(path)
    if set_new:
        new_timestamp = (stats.st_atime_ns + _TIMESTAMP_DELTA, stats.st_mtime_ns + _TIMESTAMP_DELTA)
    else:
        new_timestamp = (stats.st_atime_ns - _TIMESTAMP_DELTA, stats.st_mtime_ns - _TIMESTAMP_DELTA)
    try:
        yield
    finally:
        os.utime(path, ns=new_timestamp)


# Public Methods 

In [30]:
fp = xpu('~/cimaq_memory/cimaq_dir_list.json')

os.stat(fp)

# def json_read(fp):
#     with open(join(dname(fp), bname(fp)), 'r') as outfile:
#         json.load(outfile, cls=None, object_hook=None, parse_float=None,
#                   parse_int=None, parse_constant=None, object_pairs_hook=None)
#         outfile.close()
# fp = xpu('~/cimaq_memory/cimaq_dir_list.json')
# json.load(fp, cls=None, object_hook=None, parse_float=None,
#           parse_int=None, parse_constant=None, object_pairs_hook=None)
        
# json_read(fp)        

os.stat_result(st_mode=33204, st_ino=46792720, st_dev=2049, st_nlink=1, st_uid=1000, st_gid=1000, st_size=377, st_atime=1613319019, st_mtime=1613319019, st_ctime=1613319019)